In [ ]:
!pip install schedule
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import lxml
from lxml import html
import selenium
import io
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import schedule
import time
import http.client
import pandas as pd
from io import BytesIO
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium .webdriver.common.by import By
import time
import requests
import pandas as pd
import io
!pip install --upgrade selenium

# using chromerdrver to extract an excel file from australian agriculture gouv website

In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
import time
import requests
import pandas as pd
import io

# URL du site web 
url = "https://www.agriculture.gov.au/abares/research-topics/agricultural-outlook/data#australian-crop-report-data"

# Demarrer web driver
driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()))
# Aller sur le site web
driver.get(url)
# Attendre que la page se charge
time.sleep(5)

# XPath du fichier Excel 
xpath = '//*[@id="block-system-main-block"]/article/div/div/div[1]/div[3]/div/div/table/tbody/tr[1]/td[1]/a'

# Trouver le premier element qui correspond au XPath
element = driver.find_element(By.XPATH, xpath)

# Obtenir l'URL du fichier Excel à partir de l'attribut 'href' de l'element
excel_url = element.get_attribute('href')

# Fermer WebDriver
driver.quit()

# Telecharger le fichier Excel 
excel_response = requests.get(excel_url)

# lire le contenu du fichier Excel en tant que DataFrame
df = pd.read_excel(io.BytesIO(excel_response.content))

# Afficher le DataFrame
print(df)

In [ ]:
fichier=io.BytesIO(excel_response.content)

show tables using sheet name

In [ ]:
all_sheets = pd.read_excel(fichier, sheet_name=None)
all_sheets['Table 13']

In [ ]:
sheets_with_header_8 = ["Index", "Table 3", "Table 13"]

# Afficher mon fichier xlsx de 18 feuilles de calcul et assigner le header a chaque feuille 
all_sheets = pd.read_excel(fichier, sheet_name=None)  # Load all sheets first

for sheet in all_sheets.keys():
    if sheet in sheets_with_header_8:
        all_sheets[sheet] = pd.read_excel(fichier, sheet, header=8,index_col= 0)
    else:
        all_sheets[sheet] = pd.read_excel(fichier, sheet, header=7,index_col= 0)

all_sheets 
# Supprimer les colonnes et les ligne vides
for sheet_name in all_sheets.keys():
    all_sheets[sheet_name] = all_sheets[sheet_name].dropna(thresh=len(all_sheets[sheet_name].columns) - 2)
    all_sheets[sheet_name] = all_sheets[sheet_name].dropna(axis=1, thresh=len(all_sheets[sheet_name]) - 2)
all_sheets
# Renommer les colonne unmaned
def rename_unnamed(df):
    cols = pd.Series(df.columns)
    for i in range(1, len(cols)):
        if "unnamed" in cols[i].lower():
            cols[i] = cols[i - 1]
    df.columns = cols
    return df

for sheet_name in all_sheets.keys():
    all_sheets[sheet_name] = rename_unnamed(all_sheets[sheet_name])
# Afficher chacune des feuilles du fichier
all_sheets["Table 1"]

# Cleaning tables

In [ ]:
a = pd.read_csv('Table 3.csv', delimiter=';')

# Display the first few rows of the data
data.head()
data = data.drop(0)

# Rename the columns
data.columns = ['Year', 'NSW_Area', 'NSW_Production', 'Queensland_Area', 'Queensland_Production', 'Australia_Area', 'Australia_Production']

# Convert the data to long format
data_long = data.melt(id_vars='Year', var_name='Measure', value_name='Value')

# Split the 'Measure' column into 'Region' and 'Unit' columns
data_long[['Region', 'Unit']] = data_long['Measure'].str.split('_', expand=True)

# Drop the 'Measure' column as it is no longer needed
data_long = data_long.drop('Measure', axis=1)

# Convert 'Value' column to numeric
data_long['Value'] = pd.to_numeric(data_long['Value'])

# Display the first few rows of the transformed data
data_long.head()
data_long.to_csv("table_3.csv",sep=";",index=False)

In [ ]:
data_new=pd.read_csv('Table 4.csv',delimiter=';')
data_new = data_new.drop(0)

# Rename the columns
data_new.columns = ['Crop', 'Area', 'Yield', 'Production', 'Area Change', 'Production Change']

# Convert the data to long format
data_new_long = data_new.melt(id_vars='Crop', var_name='Measure', value_name='Value')

# Display the first few rows of the transformed data
data_new_long.head()
data_new_long.to_csv("table_4.csv",sep=';',index=False)

# saving all the tables in csv database


In [ ]:
for sheet_name, df in all_sheets.items():
    df.to_csv(f"{sheet_name}.csv",sep=';',index=False)